In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import os

# === Compact DAE Definition ===
class CompactDAE(nn.Module):
    def __init__(self, window_size):
        super(CompactDAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(window_size, window_size // 2),
            nn.ReLU(),
            nn.Linear(window_size // 2, window_size // 4),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(window_size // 4, window_size // 2),
            nn.ReLU(),
            nn.Linear(window_size // 2, window_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# === Configurations ===
window_size = 25
num_samples = 10000
test_ratio = 0.1
batch_size = 64
epochs = 100
learning_rate = 1e-3
noise_mean_min, noise_mean_max = -0.1, 0.1
noise_std_min, noise_std_max = 0.2, 0.6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Data Generation ===
def generate_dataset(num_samples, window_size):
    clean_data = []
    noisy_data = []

    for _ in range(num_samples):
        base_seq = np.random.choice([0, 1], size=window_size).astype(np.float32)

        noise_mean = np.random.uniform(noise_mean_min, noise_mean_max)
        noise_std = np.random.uniform(noise_std_min, noise_std_max)
        noise = np.random.normal(loc=noise_mean, scale=noise_std, size=window_size)

        noisy_seq = np.clip(base_seq + noise, 0, 1).astype(np.float32)
        clean_data.append(base_seq)
        noisy_data.append(noisy_seq)

    return np.array(noisy_data), np.array(clean_data)

# === Load and split dataset ===
X, Y = generate_dataset(num_samples, window_size)
split_idx = int(num_samples * (1 - test_ratio))
X_train, X_test = X[:split_idx], X[split_idx:]
Y_train, Y_test = Y[:split_idx], Y[split_idx:]

# === Dataloaders ===
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(Y_train))
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(Y_test))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

# === Model, Loss, Optimizer ===
model = CompactDAE(window_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# === Training Loop ===
print("Training Compact DAE...")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for noisy_batch, clean_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        noisy_batch = noisy_batch.to(device)
        clean_batch = clean_batch.to(device)

        optimizer.zero_grad()
        outputs = model(noisy_batch)
        loss = criterion(outputs, clean_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f}")

# === Testing Loop ===
model.eval()
total_test_loss = 0.0
with torch.no_grad():
    for noisy_batch, clean_batch in test_loader:
        noisy_batch = noisy_batch.to(device)
        clean_batch = clean_batch.to(device)

        outputs = model(noisy_batch)
        loss = criterion(outputs, clean_batch)
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_loader)
print(f"\nTest Loss = {avg_test_loss:.4f}")

# === Save Model ===
save_path = "dae_compact_trained.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")


Training Compact DAE...


Epoch 1/100: 100%|██████████| 141/141 [00:00<00:00, 171.73it/s]


Epoch 1: Train Loss = 0.2491


Epoch 2/100: 100%|██████████| 141/141 [00:00<00:00, 256.19it/s]


Epoch 2: Train Loss = 0.2393


...

Epoch 98: Train Loss = 0.2057


Epoch 99/100: 100%|██████████| 141/141 [00:00<00:00, 268.12it/s]


Epoch 99: Train Loss = 0.2057


Epoch 100/100: 100%|██████████| 141/141 [00:00<00:00, 249.98it/s]

Epoch 100: Train Loss = 0.2057

Test Loss = 0.2095
Model saved to dae_compact_trained.pth
